Provide a diagram of Hive architecture and explain its main elements.

┌────────────────────────┐
│     User Interface     │  (CLI, UI, JDBC/ODBC, Thrift)
└──────────┬─────────────┘
           │
           ▼
┌────────────────────────┐
│     Driver Layer       │
│  (Parser, Planner,     │
│   Optimizer, Executor) │
└──────────┬─────────────┘
           │
           ▼
┌────────────────────────┐
│   Metastore (DB)       │
│  (Schema, Metadata)    │
└──────────┬─────────────┘
           │
           ▼
┌────────────────────────┐
│    Execution Engine    │
│ (Compiles query into   │
│  MapReduce/Tez/Spark)  │
└──────────┬─────────────┘
           │
           ▼
┌────────────────────────┐
│   Hadoop Distributed   │
│   File System (HDFS)   │
│   (Actual Data)        │
└────────────────────────┘

1. User Interface (UI)
How users interact with Hive.
Tools:
    - CLI (Command Line Interface)
    - Beeline (JDBC client)
    - Hive Web UI
    - JDBC/ODBC drivers (for BI tools like Tableau, Power BI)
    - Thrift server (remote access)

2. Driver Layer
Controls the lifecycle of HiveQL statement execution.
Components:
    - Parser: Parses HiveQL and checks syntax.
    - Planner: Generates logical plan.
    - Optimizer: Optimizes query (joins, filters, projections).
    - Executor: Generates execution plan and coordinates with Execution Engine.

3. Metastore
- Central repository that stores metadata:
    - Table schemas
    - Column types
    - Table locations
    - Partitions
    - SerDe (Serialization/Deserialization) information
- Typically backed by RDBMS (MySQL, PostgreSQL).
- Critical — no query can run without consulting Metastore.

4. Execution Engine
- Converts optimized logical plan into physical execution:
    - MapReduce Jobs
    - Apache Tez
    - Apache Spark
- Controls actual execution of query on Hadoop cluster.
- Communicates with Hadoop YARN Resource Manager.

5. HDFS (Hadoop Distributed File System)
- The actual storage layer for data.
- Stores:
    - Table data (structured/unstructured)
    - Files: ORC, Parquet, Avro, CSV, Text
- Hive reads/writes data from HDFS.
- You can use external tables pointing to data already in HDFS.
Summary Flow: User Query → Parser → Optimizer → Execution Engine → Data fetched from HDFS → Results returned to User

Draw Spark architecture and explain its various components.

┌───────────────────────────────────┐
│         Spark Application         │
└───────────────────────────────────┘
                   │
                   ▼
┌───────────────────────────────────┐
│         Driver Program            │
│  (SparkContext, Scheduler, etc.)  │
└───────────────────────────────────┘
                   │
                   ▼
          ┌─────────────────┐
          │  Cluster Manager │  (Standalone / YARN / Mesos / Kubernetes)
          └─────────────────┘
                   │
     ┌─────────────┴─────────────┐
     ▼                           ▼
┌─────────────┐           ┌─────────────┐
│ Executor 1  │           │ Executor N  │
│ (Task slot) │    ...    │ (Task slot) │
└─────────────┘           └─────────────┘

1. Driver Program: 
The "main" program — where SparkContext is created.
Coordinates the whole Spark job.
Components inside:
    SparkContext → entry point of Spark program.
    DAG Scheduler → builds stages of tasks.
    Task Scheduler → sends tasks to Executors.
    Backend → communicates with Cluster Manager.
Runs on one machine.

2. Cluster Manager: 
Allocates resources across the cluster.
Spark supports:
    Standalone cluster manager (built-in)
    YARN (Hadoop ecosystem)
    Apache Mesos
    Kubernetes
Manages Executors on worker nodes.

3. Executors: 
Run on worker nodes.
Each Executor:
    Executes Tasks assigned by Driver.
    Has its own memory and caches.
    Reports status and results back to Driver.
Multiple Executors → parallelism.

4. Tasks & Jobs: 
Spark breaks user action (e.g. collect(), saveAsTextFile()) into Jobs.
Jobs → divided into Stages.
Stages → divided into Tasks.
Tasks → run in parallel on Executors.
Tasks are the unit of execution.

- Execution Flow
    1. User writes Spark code (RDD, DataFrame, etc.)
    2. Driver starts → creates SparkContext
    3. SparkContext contacts Cluster Manager
    4. Executors are launched on Worker Nodes
    5. Driver sends Tasks to Executors
    6. Executors run Tasks, return results to Driver
    7. Final results returned to user

Summary
Component	Role
Driver Program -	Controls application
SparkContext -	Entry point, manages job
Cluster Manager -	Allocates resources
Executors -	Run tasks in parallel
Tasks & Jobs -	Units of execution

Key Features of Spark Architecture
- In-memory computation
- Fault tolerance (DAG lineage)
- Distributed — scales horizontally
- Supports:
    - Batch processing
    - Streaming
    - SQL
    - Machine Learning
    - Graph processing


Explain partitioning (in hive) with an example.

Partitioning in Hive means dividing a large table into smaller parts (partitions) based on the values of a column.
    - It improves query performance
    - It reduces the amount of data scanned → faster queries

How it works:
    - You choose one or more columns as partition keys
    - Hive stores data in separate directories per partition value
    - When you run a query, Hive will only scan relevant partitions

Hive Query: Create Partitioned Table
CREATE TABLE sales_partitioned (
    order_id INT,
    product_name STRING,
    amount DOUBLE,
    country STRING
)
PARTITIONED BY (year INT, month INT)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
STORED AS TEXTFILE;

List any 2 differences between Coalesce and Repartition (in spark)

| Aspect            | `coalesce()`                                                              | `repartition()`                                                                 |
| ----------------- | ------------------------------------------------------------------------- | ------------------------------------------------------------------------------- |
| 1. **Shuffling** | Tries to **minimize shuffle**                                             | Performs a **full shuffle**                                                     |
| 2. **Use Case**  | Mainly used to **reduce** number of partitions (e.g. going from 100 → 10) | Can be used to **increase or decrease** partitions (e.g. 10 → 100, or 100 → 10) |

rdd2 = rdd1.coalesce(10),
rdd3 = rdd1.repartition(100)

Summary:
    - coalesce() → used to reduce partitions, minimal shuffle (efficient)
    - repartition() → increase/decrease partitions, full shuffle (more flexible, but expensive)

What is Big Data? Describe the three fundamental aspects that define it.

Big Data refers to datasets that are too large, complex, or fast-changing to be processed and analyzed using traditional data-processing tools.

In other words — Big Data is not just "a lot of data," it's data that:
    - can’t fit in a single machine
    - arrives very fast (real-time or near real-time)
    - is often messy, unstructured, or of many types

The Three Fundamental Aspects of Big Data
(often called 3 V’s of Big Data):

1. Volume (How Much Data): 
- Definition: The sheer amount of data generated every second.
- Example:
    - Facebook generates petabytes of data daily.
    - IoT sensors generate massive streams of data.
    - Challenge: Storage, distributed processing.

2. Velocity (How Fast Data Arrives)
- Definition: The speed at which new data is generated and moves around.
- Example:
    - Stock market data → milliseconds latency.
    - Real-time fraud detection → fast decision making needed.
    - Challenge: Real-time processing, streaming analytics.

3. Variety (How Many Types of Data)
- Definition: Different types of data — structured, semi-structured, unstructured.
- Example:
    - Structured → Tables in databases.
    - Semi-structured → JSON, XML.
    - Unstructured → Text, video, images, audio, social media posts.
    - Challenge: Data integration, processing diverse formats.

##### What are the key differences between a Data Lake and a Data Warehouse? 

Aspect	Data Lake	Data Warehouse
Data Type	Raw data — structured, semi-structured, unstructured	Mostly structured data (organized into tables, schema-on-write)
Schema	Schema-on-read → apply schema when you read the data	Schema-on-write → schema defined before data is written
Purpose	Data exploration, advanced analytics, machine learning	Business reporting, BI dashboards, historical analysis
Data Storage	Low-cost storage (HDFS, S3, Blob Storage)	High-performance storage (specialized databases like Snowflake, Redshift, BigQuery)
Users	Data scientists, data engineers	Business analysts, BI professionals
Data Processing	Batch + Real-time (with Spark, Flink, etc.)	Mostly batch (SQL-based)
Cost	Cheaper (store all data, no need to clean before storage)	More expensive (optimized, clean, curated data)
Flexibility	Very flexible — any type of data	Less flexible — structured data only
Examples	Hadoop, Amazon S3 Data Lake, Azure Data Lake	Snowflake, Google BigQuery, Amazon Redshift, Teradata

- Summary:
    - Data Lake = store everything → flexible → good for AI/ML → cheaper
    - Data Warehouse = store cleaned, structured data → good for business reporting → faster queries

- When to use:
Use Case	Recommended
Store all raw data from many sources	Data Lake
Build BI reports for CEO & management	Data Warehouse
Train machine learning models on image/text/audio data	Data Lake
Run complex SQL queries with guaranteed fast performance	Data Warehouse

- Conclusion:
    - Data Lakes are flexible, cheap, great for ML
    - Data Warehouses are optimized for fast BI reporting


What are the different types of NoSQL databases? Write a brief note.

Types of NoSQL Databases — A Quick Guide
NoSQL = "Not Only SQL" → designed for:
- High scalability
- Handling unstructured or semi-structured data
- Flexibility → schema-less

There are 4 main types of NoSQL databases:

1. Key-Value Stores
Data stored as key-value pairs (like a dictionary or hashmap)
Best for:
Simple lookups
Session management
Caching
User profile storage
Examples:
Redis
Amazon DynamoDB
Memcached

2. Document Stores
Data stored as documents (typically JSON, BSON, or XML)
Documents can have nested structures and are schema-less
Best for:
Content management
Product catalogs
Mobile apps
Event logging
Examples:
MongoDB
CouchDB
Amazon DocumentDB

3. Column-Family Stores
Data stored in columns, not rows
Very good for highly scalable, distributed systems
Best for:
Time-series data
Analytical queries
Event logging
Large datasets across multiple machines
Examples:
Apache Cassandra
Apache HBase
ScyllaDB

4. Graph Databases
Data stored as nodes and edges
Best for modeling relationships (networks, social graphs, hierarchies)
Best for:
Social networks
Recommendation engines
Fraud detection
Knowledge graphs
Examples:
Neo4j
Amazon Neptune
ArangoDB
TigerGraph

- Summary Table:
Type	Data Model	Example Use Case	Example DBs
Key-Value Store	Key → Value	Caching, session store	Redis, DynamoDB
Document Store	Document (JSON/BSON)	Content mgmt, catalogs	MongoDB, CouchDB
Column-Family	Column families	Time-series, analytics	Cassandra, HBase
Graph Database	Nodes + Edges	Social networks, fraud	Neo4j, Neptune

- Final Takeaway:
    - NoSQL = flexible, scalable, handles Big Data and unstructured data
    - Choice depends on use case:
Simple key lookups → Key-Value
Flexible docs → Document DB
Fast analytical queries → Column-Family
Complex relationships → Graph DB

What are the main features of Apache Kafka that make it a popular choice for real-time data
streaming?

1. High Throughput
    - Kafka can process millions of messages per second
    - Handles large data volumes easily
    - Suitable for both batch and real-time streaming

2. Scalability
    - Kafka is horizontally scalable → just add more brokers (servers)
    - Supports partitioning of topics → enables parallelism

3. Fault Tolerance
    - Kafka is a distributed system → data is replicated across multiple brokers
    - If one broker fails → others continue
    - Ensures high availability and durability

4. Durability
    - Kafka persists messages to disk
    - Messages can be retained for a configurable retention period
    - Supports long-term replay of data

5. Real-time Stream Processing
    - Kafka integrates with stream processing frameworks:
        - Apache Spark Streaming
        - Apache Flink
        - Kafka Streams API (built-in lightweight stream processing)
    - Enables real-time analytics

6. Publish-Subscribe Model
    - Kafka is a Pub-Sub system:
        - Producers publish data
        - Consumers subscribe and consume data
    - Supports:
        - One-to-many delivery
        - Multiple consumers can read same stream independently

Feature	Benefit
High Throughput - Process millions of messages/sec
Scalability -	Add brokers and partitions easily
Fault Tolerance -	Data replicated → high availability
Durability -	Messages persisted to disk
Real-time Processing -	Instant streaming and analytics
Pub-Sub Model -	Easy decoupling of producers/consumers
Exactly Once -	Reliable, accurate processing
Large Ecosystem -	Easy to integrate with other tools

- Conclusion
    - Apache Kafka is fast, scalable, reliable, fault-tolerant → that’s why it’s widely used for:
        - Real-time analytics
        - Event-driven architectures
        - Data pipelines
        - Streaming ETL

##### HDFS Commands

Print directory with details: 
hdfs dfs -ls -h InputDir

Upload multiple files: 
hdfs dfs -put *.txt XYZ

Display file line by line: 
hdfs dfs -cat file.txt | less

Recursive delete: 
hdfs dfs -rm -r SampleDir

Copy from HDFS to local: 
hdfs dfs -get testfile ./

Copy file1.txt to OutputDir as file2.txt: 
hdfs dfs -cp InputDir/file1.txt OutputDir/file2.txt

Delete empty directory XYZ: 
hdfs dfs -rmdir XYZ

Recursively list files under SampleDir: 
hdfs dfs -ls -R SampleDir

Change permission of file.txt to read-only (444): 
hdfs dfs -chmod 444 file.txt

##### Apache Spark RDD Operations

Create RDD: 
rdd = sc.parallelize([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

Print first 4 elements: 
print(rdd.take(4))

Print first element: 
print(rdd.first())

Load text file into RDD: 
file_rdd = sc.textFile("/path/to/file.txt")

Number of elements: 
num_elements = rdd.count()
print(f"Number of elements in RDD: {num_elements}")

Sum of elements: 
sum_elements = rdd.reduce(lambda x, y: x + y)
print(f"Sum of all elements in RDD: {sum_elements}")

map → applies a function to each element of the RDD
Returns a new RDD with transformed elements

Example: Multiply each element by 2: 
mapped_rdd = rdd.map(lambda x: x * 2)
print(mapped_rdd.collect())
Explanation:

filter → keeps only elements that satisfy a condition
Returns a new RDD with filtered elements

Example: Keep only even numbers: 
filtered_rdd = rdd.filter(lambda x: x % 2 == 0)
print(filtered_rdd.collect())


##### Hive Queries

Create database: 
CREATE DATABASE emp;

Load data from local file: 
LOAD DATA LOCAL INPATH '/path/to/your/file.csv'
INTO TABLE orders1;

Create external table: 
USE emp;
CREATE EXTERNAL TABLE employee (
    emp_id INT,
    name STRING,
    location STRING,
    dep STRING,
    designation STRING,
    salary DOUBLE
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
STORED AS TEXTFILE
LOCATION '/user/hive/warehouse/emp/employee';

Inner join on Table1 and Table2: 
SELECT t1.*, t2.*
FROM Table1 t1
INNER JOIN Table2 t2
ON t1.id = t2.id;

average salary of employees based on location: 
SELECT location, AVG(salary) AS avg_salary FROM employee GROUP BY location;

##### mongoDB

Create collection: 
db.createCollection("product_collection")

Insert documents: 
db.product_collection.insertMany([
    { name: "AC", rating: 5, brand: "Samsung" },
    { name: "Laptop", rating: 4, brand: "Dell" },
    { name: "Mobile", rating: 5, brand: "Apple" },
    { name: "TV", rating: 3, brand: "Sony" },
    { name: "Refrigerator", rating: 5, brand: "LG" }
])

Find products with 5/5 rating: 
db.product_collection.find({ rating: 5 })

Update name 'AC' → 'Air conditioner': 
db.product_collection.updateMany(
    { name: "AC" },
    { $set: { name: "Air conditioner" } }
)

Print updated record: 
db.product_collection.find({ name: "Air conditioner" })

Fetch users with age >= 34: 
db.users.find({ age: { $gte: 34 } })

Delete user Ravi: 
db.users.deleteOne({ name: "Ravi" })


Load dataset: 
df = spark.read.csv("/path/to/qs_world_university_rankings.csv", header=True, inferSchema=True)

Count distinct Institutions: 
num_institutions = df.select("Institution").distinct().count()
print(f"Number of Institutions: {num_institutions}")

Institutions from India: 
num_india_institutions = df.filter(df["Location Full"] == "India").select("Institution").distinct().count()
print(f"Number of Institutions from India: {num_india_institutions}")

Average Citations per Faculty for India: 
df.filter(df["Location Full"] == "India").selectExpr("avg(`Citations per Faculty`) as avg_citations").show()

Institutions with 100% International Students: 
from pyspark.sql.functions import regexp_replace, col
df = df.withColumn("International Students", regexp_replace(col("International Students"), "%", "").cast("double"))
df.filter(col("International Students") == 100).select("Institution", "Location Full").show(truncate=False)

#### 1. What’s the overall minimum, maximum and average salary from the dataset? ( 6 marks)

In [ ]:
spark = SparkSession.builder.appName("SalaryAnalysis").getOrCreate()
file_path = "path/to/mba_placement.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)
df.printSchema()
df.show(5)
columns = df.columns
print("Columns:", columns)
possible_salary_columns = [col for col in columns if 'salary' in col.lower()]
print("Possible salary columns:", possible_salary_columns)
if possible_salary_columns:
    salary_col = possible_salary_columns[0]
    stats = df.select(
        min(salary_col).alias("min_salary"),
        max(salary_col).alias("max_salary"),
        avg(salary_col).alias("avg_salary")
    )
    stats.show()
else:
    print(f"No salary column found. Columns: {columns}")

#### 2. How many  female candidate are not placed ?  ( 4 marks)

In [ ]:
female_not_placed_count = df.filter(
    (col("gender") == "F") & 
    (col("status") == "Not Placed")
).count()
print(f"Number of female candidates not placed: {female_not_placed_count}")

#### `3`. Out of total male candidates placed, how many does not have any work experience ? (3marks)

In [ ]:
male_placed_df = df.filter(
    (lower(trim(col("gender"))) == "m") &
    (lower(trim(col("status"))) == "placed")
)
male_placed_no_workex_count = male_placed_df.filter(
    (lower(trim(col("workex"))) == "no")
).count()
total_male_placed = male_placed_df.count()
print(f"Total male candidates placed: {total_male_placed}")
print(f"Number of male candidates placed with NO work experience: {male_placed_no_workex_count}")

#### 4. Remove the feature 'sl_no' and also remove null values from the DataFrame. (2 marks)  

In [ ]:
df_cleaned = df.drop('sl_no')
df_cleaned = df_cleaned.na.drop()

### 1.	Convert all string columns into numeric values using StringIndexer transformer and make sure now DataFrame does not have any string columns anymore. (5 marks)

In [ ]:
string_cols = [field.name for field in df.schema.fields if isinstance(field.dataType, StringType)]
print(f"String columns to index: {string_cols}")
from pyspark.ml import Pipeline
indexers = [
    StringIndexer(inputCol=col, outputCol=f"{col}_indexed", handleInvalid='keep')
    for col in string_cols
]
pipeline = Pipeline(stages=indexers)
df_indexed = pipeline.fit(df).transform(df)
df_final = df_indexed.drop(*string_cols)
df_final.show(5)
final_string_cols = [field.name for field in df_final.schema.fields if isinstance(field.dataType, StringType)]
print(f"Remaining string columns: {final_string_cols}")
final_row_count = df_final.count()
print(f"Total rows in final DataFrame: {final_row_count}")

### 2. Using vectorAssembler combines all columns (except target column i.e., 'salary') of spark DataFrame into single column (name as features). Make sure DataFrame now contains only two columns features and salary. ( 5 marks)

In [ ]:
feature_cols = [col for col in df_numeric.columns if col != 'salary']
print(f"Feature columns: {feature_cols}")
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
df_final = assembler.transform(df_numeric)
df_final = df_final.select("features", "salary")
print("Final DataFrame with features and salary:")
df_final.show(5, truncate=False)
df_final.printSchema()

### 3 .	Split the vectorized dataframe into training and test sets with one fourth records being held for testing (**3** marks)

In [ ]:
train_df, test_df = df_final.randomSplit([0.75, 0.25], seed=42)
print(f"Training set count: {train_df.count()}")
print(f"Test set count: {test_df.count()}")
print("Training Set Sample:")
train_df.show(5, truncate=False)
print("Test Set Sample:")
test_df.show(5, truncate=False)

### 4.	Build a LinearRegression model on train set  use featuresCol="features" and  'salary'(6 marks)

In [ ]:
from pyspark.sql.types import StringType
lr = LinearRegression(featuresCol="features", labelCol="salary")
lr_model = lr.fit(train_df)
print(f"Coefficients: {lr_model.coefficients}")
print(f"Intercept: {lr_model.intercept}")
test_results = lr_model.evaluate(test_df)
print(f"RMSE (Root Mean Squared Error): {test_results.rootMeanSquaredError}")
print(f"MAE (Mean Absolute Error): {test_results.meanAbsoluteError}")
print(f"R2 (R-squared): {test_results.r2}")
spark.stop()

### 5. Perform prediction on the testing data and Print MSE value? (6 marks)

In [ ]:
predictions = lr_model.transform(test_df)
predictions.select("features", "salary", "prediction").show(5, truncate=False)
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(
    labelCol="salary", predictionCol="prediction", metricName="mse"
)
mse = evaluator.evaluate(predictions)
print(f"Mean Squared Error (MSE) on test data: {mse}")
predictions.select("features", "salary", "prediction").show(5, truncate=False)